<a href="https://colab.research.google.com/github/suna0107/ANN_DL101/blob/main/(250404)classification_fc%2BRelu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 기본 FC + ReLU 구조
- 데이터 불러오기 & label 매핑& 불균형 데이터 처리
- `Train / Validation / Test` 분리
- `Dataset` & `DataLoader` 정의
- 모델 정의 (`BasicFCModel`)
- pos_weight 계산
- 학습 루프
- 최종 Test 평가 (Accuracy, Precision, Recall, F1, Confusion Matrix)

##1. 데이터 불러오기 & label 매핑 & 불균형 데이터 처리

In [17]:
import pandas as pd
import numpy as np
import torch
from collections import Counter

item_df = pd.read_pickle("item_input.pkl")
type(item_df['input_vec'].iloc[0])  # 👉 numpy.ndarray (예상)
len(item_df['input_vec'].iloc[0])   # 👉 1568차원

2336

In [18]:
# 2. label 컬럼 생성 ('해당' → 1, '비해당' → 0)
item_df['label'] = item_df['결과'].map({'해당': 1, '비해당': 0}).astype(int)


In [20]:
item_df['label']

,label
0,0
1,0
2,0
3,0
4,0
...,...
2799,0
2800,0
2801,0
2802,0


In [22]:
# 4. input & label 추출
#  np.stack() : 2D 배열로 만들기 위함 ->X.shape = (2865, 2336)= (샘플 수, 벡터 차원)
X = np.stack(item_df['input_vec'].values)
y = item_df['label'].values


In [25]:
print("🧩 X shape:", X.shape)  # → (샘플 수, 1568) 예상
print("🎯 y shape:", y.shape)  # → (샘플 수,) 예상

🧩 X shape: (2804, 2336)
🎯 y shape: (2804,)


- 불균형 클래스 -> pos_weight 적용

In [26]:
# 5. 클래스 비율 확인 & pos_weight 계산
from collections import Counter

counter = Counter(y) # 클래스 비율 확인
print("📊 클래스 분포:", counter)  # 예: Counter({0: 2439, 1: 365})



📊 라벨 분포: Counter({np.int64(0): 2480, np.int64(1): 324})


In [30]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

neg = counter[0]  # 비해당
pos = counter[1]  # 해당

pos_weight = torch.tensor(neg / pos).to(device) # neg / pos= 희귀 클래스 가중치 계산
# 7.65
#손실 함수에서 클래스 1이 틀릴 경우 약 7.65배 더 큰 손실을 주겠다 라는 뜻이야.

print(f"🔧 적용할 pos_weight: {pos_weight:.4f}")

🔧 적용할 pos_weight: 7.6543


## 2. 데이터 분리: Train / Val / Test

- 추후 결과 분석을 위해 idex 따로 저장
- Stratify=y
- 클래스 불균형 상태기에 클래스 0 / 1 비율을 유지해줘야함

In [61]:
# ✅ 1. 전체 인덱스 → test 인덱스만 추출
all_indices = np.arange(len(X))

_, idx_test = train_test_split(
    all_indices, test_size=0.2, random_state=42, stratify=y
)

In [31]:
from sklearn.model_selection import train_test_split

# 1. 먼저 전체에서 train+val (80%) vs test (20%) 분리
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2. 다시 train+val을 train (80%) vs val (20%)로 분리 (→ 64/16/20)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42, stratify=y_temp
)

In [32]:
print("🟢 Train:", X_train.shape, y_train.shape)
print("🟡 Val:  ", X_val.shape, y_val.shape)
print("🔵 Test: ", X_test.shape, y_test.shape)


🟢 Train: (1794, 2336) (1794,)
🟡 Val:   (449, 2336) (449,)
🔵 Test:  (561, 2336) (561,)


## 3. Dataset & DataLoader 정의

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class ItemDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(ItemDataset(X_train, y_train), batch_size=64, shuffle=True)
val_loader = DataLoader(ItemDataset(X_val, y_val), batch_size=64)
test_loader = DataLoader(ItemDataset(X_test, y_test), batch_size=64)

In [34]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# 1. 넘파이 배열 → 텐서로 변환 (float32, label은 float)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)

X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# 2. TensorDataset으로 묶기
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# 3. DataLoader로 감싸기
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [35]:
print("✅ 학습 배치 수:", len(train_loader)) # 총샘플 수/ 배치 크기(64)1794 ÷ 64 ≈ 28.0 → 29개 (마지막 배치에 2개 포함)
print("✅ 검증 배치 수:", len(val_loader))
print("✅ 테스트 배치 수:", len(test_loader))


✅ 학습 배치 수: 29
✅ 검증 배치 수: 8
✅ 테스트 배치 수: 9


## 4. 모델 정의: BasicFCModel

In [46]:
class BasicFCModel(nn.Module):
    def __init__(self):
        super(BasicFCModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(2336, 1024),     # 입력: 2336차원 → 은닉층 1024차원
            nn.ReLU(),                 # 비선형성 추가
            nn.Linear(1024, 512),      # 2층: 1024 → 512
            nn.ReLU(),
            nn.Dropout(0.3),           # 30% 드롭아웃 → 과적합 방지
            nn.Linear(512, 128),       # 3층: 512 → 128
            nn.ReLU(),
            nn.Linear(128, 1),         # 마지막 출력층: 1개 노드 (이진 분류)
        )
    def forward(self, x):
        return self.net(x)

## 5. 학습 및 검증 루프

### 1. 손실 함수, 옵티마이저, 모델 선언

- scheduler.step()  : 학습률 점진적으로 감소시켜 성능 안정화
- early stopping : 검증 성능 개선 없을 시 학습 자동 중단
- best model 저장: 최고 성능 모델 저장 후 추후 테스트에 활용

In [47]:
import torch.nn as nn
import torch.optim as optim

# 1. 모델 정의
model = BasicFCModel()  # ✅ 모델 먼저 정의해야 함
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2. 이진 분류이므로 마지막에 Sigmoid → BCELoss 사용
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

# 3. 옵티마이저 & 스케줄러
optimizer = optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)  # 5 epoch마다 LR 절반으로


# 4. Early Stopping 변수
best_val_f1 = 0
patience = 3
patience_counter = 0

### 2. 학습 루프 함수 정의

In [48]:
# 3. 학습 루프
def train_epoch(loader):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

### 3. 검증 함수 (Sigmoid 적용 추가!)

In [49]:
# 4. 검증 루프 (accuracy, precision, recall, f1)
def eval_epoch(loader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
            logits = model(X_batch)               # → logit
            probs = torch.sigmoid(logits)         # ✅ 확률로 변환
            preds = (probs > 0.5).float()         # threshold 적용
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())

    # 메트릭 계산
    all_preds = np.array(all_preds).astype(int)
    all_labels = np.array(all_labels).astype(int)

    acc = (all_preds == all_labels).mean()
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    return acc, precision, recall, f1


## 6. 학습 실행

In [55]:
# 5. 전체 학습 과정
from sklearn.metrics import precision_score, recall_score, f1_score

for epoch in range(1, 21):
    train_loss = train_epoch(train_loader)
    val_acc, val_prec, val_rec, val_f1 = eval_epoch(val_loader)
    scheduler.step()  # 학습률 조절

    print(f"📘 Epoch {epoch:02d} | Train Loss: {train_loss:.4f} | "
          f"Val Acc: {val_acc:.4f} | P: {val_prec:.4f} | R: {val_rec:.4f} | F1: {val_f1:.4f}")

     # Best model 저장
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        torch.save(model.state_dict(), "best_model.pt")
        patience_counter = 0
        print("📌 Best model saved.")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print("⛔ Early stopping triggered!")
            break

📘 Epoch 01 | Train Loss: 0.1310 | Val Acc: 0.9131 | P: 0.5867 | R: 0.8462 | F1: 0.6929
📌 Best model saved.
📘 Epoch 02 | Train Loss: 0.1281 | Val Acc: 0.9131 | P: 0.5844 | R: 0.8654 | F1: 0.6977
📌 Best model saved.
📘 Epoch 03 | Train Loss: 0.1251 | Val Acc: 0.9109 | P: 0.5769 | R: 0.8654 | F1: 0.6923
📘 Epoch 04 | Train Loss: 0.1264 | Val Acc: 0.9131 | P: 0.5844 | R: 0.8654 | F1: 0.6977
📘 Epoch 05 | Train Loss: 0.1211 | Val Acc: 0.9154 | P: 0.5946 | R: 0.8462 | F1: 0.6984
📌 Best model saved.
📘 Epoch 06 | Train Loss: 0.1197 | Val Acc: 0.9131 | P: 0.5844 | R: 0.8654 | F1: 0.6977
📘 Epoch 07 | Train Loss: 0.1386 | Val Acc: 0.9154 | P: 0.5946 | R: 0.8462 | F1: 0.6984
📘 Epoch 08 | Train Loss: 0.1263 | Val Acc: 0.9131 | P: 0.5844 | R: 0.8654 | F1: 0.6977
⛔ Early stopping triggered!


IndentationError: unexpected indent (<ipython-input-54-b9d6344a7ba6>, line 2)

##7.Test

##1. 저장된 Best 모델 불러오기

In [56]:
# 1. 모델 객체를 다시 선언하고 weight 로드
model = BasicFCModel().to(device)## 주의사항 : BasicFCModel 클래스는 앞에서 정의한 Sigmoid 없는 버전이어야 해!
model.load_state_dict(torch.load("best_model.pt")) # best model 불러오기
model.eval()  # 꼭 평가 모드로 바꿔줘야 함


BasicFCModel(
  (net): Sequential(
    (0): Linear(in_features=2336, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.3, inplace=False)
    (5): Linear(in_features=512, out_features=128, bias=True)
    (6): ReLU()
    (7): Linear(in_features=128, out_features=1, bias=True)
  )
)

##  2. test_loader 평가 (정확도, 정밀도, 재현율, F1)

In [57]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import torch

all_preds, all_labels = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device).unsqueeze(1)

        logits = model(X_batch)
        probs = torch.sigmoid(logits)            # BCEWithLogitsLoss → Sigmoid 필요
        preds = (probs > 0.5).float()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

# numpy로 변환
all_preds = np.array(all_preds).astype(int)
all_labels = np.array(all_labels).astype(int)

# 지표 계산
acc = accuracy_score(all_labels, all_preds)
prec = precision_score(all_labels, all_preds)
rec = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"✅ 테스트 성능:")
print(f"Accuracy:  {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall:    {rec:.4f}")
print(f"F1 Score:  {f1:.4f}")


✅ 테스트 성능:
Accuracy:  0.9251
Precision: 0.6322
Recall:    0.8462
F1 Score:  0.7237


#2. 결과에 대한 해석, 원인 파악

# 1. 잘못 예측한 샘플 보기 (선택사항)

In [58]:
# 틀린 샘플 인덱스 추출
wrong_indices = np.where(all_preds != all_labels)[0]

# 몇 개 출력해보기 (예: 5개)
print("\n🔍 잘못 예측한 샘플 (예시):")
for i in wrong_indices[:5]:
    print(f"예측: {all_preds[i]}, 실제: {all_labels[i]}")



🔍 잘못 예측한 샘플 (예시):
예측: [0], 실제: [1]
예측: [1], 실제: [0]
예측: [0], 실제: [1]
예측: [0], 실제: [1]
예측: [1], 실제: [0]


## 틀린 예측을 item_df의 실제 물품 정보와 매칭

In [63]:
# 테스트셋 인덱스를 item_df에서 복구
all_indices = np.arange(len(X))

# test 인덱스 재분할 ( 동일하게 stratify 포함!)
_, idx_test = train_test_split(all_indices, test_size=0.2, random_state=42, stratify=y)

# 틀린 예측 인덱스
wrong_idx = np.where(all_preds != all_labels)[0]



                            물품명                       모델명  \
1165               PHOTO RESIST                 SRED-0005   
1531                 AUTO VALVE  AMDZ13R-X0022-4-FL619559   
2020  Inertial Measurement Unit                   FI 200P   
301             C9200L-48P-4G-A           C9200L-48P-4G-A   
1948    분전반(Distribution Panel)     DP-AC-1P75, DP-DC-100   

                                                     규격          회사명  예측값  실제값  
1165                                  1GAL GLASS BOTTLE   주식회사 동진쎄미켐    0    1  
1531  Part3R, N.C형, 1/4"(6.35mm), Super300, 센서부착형, 하...    엘에스이 주식회사    1    0  
2020  1) Gyro Operating Range:  ±1,000 º/sec  (자이로 선...     (주)파이버프로    0    1  
301   Catalyst 9300 48-port 1G copper with modular u...  주식회사 이테크시스템    0    1  
1948  DP-AC-1P75 (단상 2선 230~240V, 75kVA), DP-DC-100 ...      (주)지오닉스    1    0  
                            물품명                       모델명  \
1165               PHOTO RESIST                 SRED-0005   
1531                 AUTO

In [66]:
# 원본 item_df에서 틀린 샘플 정보 추출
wrong_items = item_df.iloc[idx_test[wrong_idx]].copy()
wrong_items['예측값'] = all_preds[wrong_idx]
wrong_items['실제값'] = all_labels[wrong_idx]

# 확인
 # (행 개수, 열 개수)
print(wrong_items[['물품명', '모델명', '규격', '회사명', '예측값', '실제값']].head(5))

                            물품명                       모델명  \
1165               PHOTO RESIST                 SRED-0005   
1531                 AUTO VALVE  AMDZ13R-X0022-4-FL619559   
2020  Inertial Measurement Unit                   FI 200P   
301             C9200L-48P-4G-A           C9200L-48P-4G-A   
1948    분전반(Distribution Panel)     DP-AC-1P75, DP-DC-100   

                                                     규격          회사명  예측값  실제값  
1165                                  1GAL GLASS BOTTLE   주식회사 동진쎄미켐    0    1  
1531  Part3R, N.C형, 1/4"(6.35mm), Super300, 센서부착형, 하...    엘에스이 주식회사    1    0  
2020  1) Gyro Operating Range:  ±1,000 º/sec  (자이로 선...     (주)파이버프로    0    1  
301   Catalyst 9300 48-port 1G copper with modular u...  주식회사 이테크시스템    0    1  
1948  DP-AC-1P75 (단상 2선 230~240V, 75kVA), DP-DC-100 ...      (주)지오닉스    1    0  


In [67]:
wrong_items.shape

(42, 17)

In [68]:
wrong_items['item_text_len'] = wrong_items['item_text'].str.len()
print(wrong_items[['item_text_len']].describe())


       item_text_len
count      42.000000
mean      122.785714
std        64.702899
min        43.000000
25%        88.250000
50%       114.500000
75%       144.250000
max       407.000000
